## Packages

In [1]:
import pandas as pd
import numpy as np
import random
import os
from scipy.stats import norm
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from scipy import optimize
import sympy
from sympy import integrate
from sympy.solvers import solve
import warnings
import time
from datetime import datetime
warnings.filterwarnings('ignore')

## HyperParameters

In [2]:
q = 0.8867
Underlying = 'SHSZ300'
N = 5   #number of lognormals
sampleDeltaChoice = [0,2,4,6,8]  #选择几个delta作为模型训练
totalTrials = 100

## Functions

In [3]:
def StrikeFromFwdDelta(CallPutFlag, Delta, F, Sigma, T):
    if CallPutFlag == 1:
        w  = 1
    else:
        w = -1
    
    K = F * np.exp(-w * Sigma * np.sqrt(T) * norm.ppf(w * Delta) + 0.5 * Sigma * Sigma * T)
    
    return K

def BS(S0, K, r, T, Sigma, q, call = True):
    d1 = (np.log(S0/K) + (r -q + 0.5 * Sigma**2) * T) / (Sigma * np.sqrt(T))
    d2 = d1 - Sigma * np.sqrt(T)
    if call:
        priceBS = np.exp(-q * T) * S0 * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)
    else:
        priceBS = -np.exp(-q * T) * S0 * norm.cdf(-d1) + np.exp(-r * T) * K * norm.cdf(-d2)
    
    return priceBS

def erfMC(a,b):
    a = 0
    b = b
    t = np.random.rand(1000000)
    z = a + (b - a) * t
    ht = (b - a) * np.exp(- z ** 2)
    erf = 2 / np.sqrt(np.pi) * np.mean(ht)
    return erf
    
def NormalCDF(x, mu = 0, sigma = 1):
    CDF = 0.5 * (1 + erfMC(a = 0, b = (x - mu) / sigma / np.sqrt(2)))
    return CDF
    

def SolveIV(S0, K, r, T, q,modelPrice, call = True, ):
    def f(Sigma):
        d1 = (np.log(S0/K) + (r -q + 0.5 * Sigma**2) * T) / (Sigma * np.sqrt(T))
        d2 = d1 - Sigma * np.sqrt(T)
        #print(d1, d2)
        if call:
            priceBS = np.exp(-q * T) * S0 * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)
        else:
            priceBS = -np.exp(-q * T) * S0 * norm.cdf(-d1) + np.exp(-r * T) * K * norm.cdf(-d2)
        #print(priceBS)
        #IV = solve(priceBS, Sigma)
        return (priceBS - modelPrice)
    IV = optimize.bisect(f,-10,10)
    return IV

def GetImpliedDividendYield(S0, r, T, Fwd):
    q = np.log(S0 * np.exp(r * T) / Fwd) / T
    return q

def ModelPrice(params, date, K, call = True):
    pT = params[:N]
    eplisonT = params[N:(2*N)]
    SigmaT = params[(2*N):(3*N)]
    QT = params[-1]
    #print(pT,eplisonT,SigmaT,QT)
    
    fwd = fwdMap[date]
    T = ((date - valuationDate).days)/365
    r = interpolate_riskFree(T)
    #q = GetImpliedDividendYield(spotPrice, r, T, fwd)
    modelPrice = 0
    if call:
        for i in range(N):
            modelPrice += pT[i] * BS(eplisonT[i] * spotPrice, K, r, T, SigmaT[i]/np.sqrt(T), q, call = True )
    else:
        for i in range(N):
            modelPrice += pT[i] * BS(eplisonT[i] * spotPrice, K, r, T, SigmaT[i]/np.sqrt(T), q, call = False) + QT * K * np.exp(-r * T)

    return modelPrice

def Error(x, y, method = 0):
    
    if method == 0:
        error = (x - y) ** 2
    else:
        error = ((x - y) / y) ** 2
        
    return error
        

def AllStrikeError(params, date, mktOptionPrice_date):
    error = 0
    for i, column in enumerate(mktOptionPrice_date.columns):
        delta = column[0]
        mktStrike = column[1]
        mktOptionPrice = mktOptionPrice_date.iloc[:,i].values
        modelPrice = ModelPrice(params, date, mktStrike, call = (delta>0))
        error += Error(modelPrice, mktOptionPrice, method = 1)
        #print(delta, mktStrike)
    error = np.sqrt(error / len(sampleDeltaChoice))
    return error[0]

## Optimization

In [4]:
def Objective(params, mktOptionPrice_date, date, returnType = 0):
    def Objective2(params):
        mktOptionPrice_date_sample = mktOptionPrice_date.iloc[:,sampleDeltaChoice]
        error = 0
        for i, column in enumerate(mktOptionPrice_date_sample.columns):
            delta = column[0]
            mktStrike = column[1]
            mktOptionPrice = mktOptionPrice_date_sample.iloc[:,i].values
            modelPrice = ModelPrice(params, date, mktStrike, call = (delta>0))
            #print(modelPrice)
            error += Error(modelPrice, mktOptionPrice, method = 1)
            #print(delta, mktStrike)
        error = np.sqrt(error / len(sampleDeltaChoice))
        return error[0]
    
    if returnType == 0:
        return Objective2
    else :
        return Objective2(params)

def Constraints(params, returnType = 0):

    def Constraints1(params):
        pT = params[:N]
        eplisonT = params[N:(2*N)]

        return np.dot(pT, eplisonT) - 1

    def Constraints2(params):
        pT = params[:N]
        QT = params[-1]
        return sum(pT) + QT - 1

    cons = [{'type':"eq",'fun':Constraints1},
           {'type':"eq",'fun':Constraints2}]
    
    if returnType == 0:
        return cons
    else :
        return Constraints1(params),Constraints2(params)

def Bounds(PARAMS, maturityChoice):
    bounds = []
    for i in range(N):
        bounds.append((0,1))
    for i in range(N):
        bounds.append((0,10))
        
    if maturityChoice == 0:
        for i in range(N):
            bounds.append((0,10))
        bounds.append((0,1))
    else:
        paramsPrevios = PARAMS[maturityChoice - 1]
        SigmaT = paramsPrevios[(2*N):(3*N)]
        QT = paramsPrevios[-1]
        
        for i in range(N):
            bounds.append((SigmaT[i],10))
        bounds.append((QT,1))   
        
    bounds = tuple(bounds)
    
    return bounds

## Read Data

In [5]:
root = os.getcwd()
input_path = os.path.join(root,'input')
output_path = os.path.join(root,'output')
filename = f'data_{Underlying}.xlsx'
file_path = os.path.join(input_path,filename)

In [6]:
data = pd.read_excel(file_path,sheet_name='data')

info = pd.read_excel(file_path,sheet_name='info',header = None)
valuationDate = pd.to_datetime(str(int(info.iloc[0,1])))
spotPrice = info.iloc[1,1]

deltaMap = pd.read_excel(file_path,sheet_name='deltaMap')
riskFree = pd.read_excel(file_path,sheet_name='riskFree')
deltaMap = deltaMap.set_index('name')['delta'].to_dict()
columns = list(data.columns.map(deltaMap))
columns[:2] = ['date','Fwd']
data.columns = columns
data['Fwd'].replace(' ',np.nan, inplace = True)
data.fillna(method = 'ffill',inplace = True)
data.set_index('date',inplace = True)

#data

## Get Market Data

In [7]:
fwdMap = data['Fwd'].to_dict()
dateList = list(data.index.unique())

mktImpliedVol = data.iloc[::2,1:]
mktFwdPrice = data.iloc[1::2,1:]
mktOptionPrice = data.iloc[1::2,1:]
mktOptionPrice.iloc[:,:] = np.nan

In [8]:
#Risk Free interpolate
x = riskFree['Date'].values
y = riskFree['Zero Rate'].values/100
#y = np.log(1 + y)
interpolate_riskFree = interp1d(x, y)


In [9]:
mktImpliedVol_date_List = []
mktFwdPrice_date_List = []
mktOptionPrice_date_List = []
mktStrikeDict={}
for date in dateList:
    mktStrikeList = []
    
    
    mktOptionPrice_date = mktOptionPrice.iloc[mktOptionPrice.index == date]
    mktFwdPrice_date = mktFwdPrice.iloc[mktFwdPrice.index == date]
    mktImpliedVol_date = mktImpliedVol.iloc[mktImpliedVol.index == date]
    
    fwd = fwdMap[date]
    T = ((date - valuationDate).days)/365
    r = interpolate_riskFree(T)
    #q = GetImpliedDividendYield(spotPrice, r, T, fwd)
    
    for Delta in mktImpliedVol.columns:

        Sigma = mktImpliedVol.loc[date,Delta]
        #print(Sigma)
        mktStrike = StrikeFromFwdDelta(np.sign(Delta), Delta, fwd, Sigma, T)
        #print(mktStrike)
        mktStrikeList.append(mktStrike)
        
        priceBS = BS(spotPrice, mktStrike, r, T, Sigma, q, call = (Delta > 0))
        #print(priceBS)
        mktOptionPrice.loc[date, Delta] = priceBS
        mktOptionPrice_date.loc[date, Delta] = priceBS
        
    #print(mktStrikeList)
    columnArray = np.array([list(mktImpliedVol_date.columns),mktStrikeList])
    #print(columnArray)
    
    mktStrikeDict[date] = mktStrikeList
    
    mktImpliedVol_date.columns = pd.MultiIndex.from_arrays(columnArray, names = ['Delta','mktStrike'])
    mktFwdPrice_date.columns = pd.MultiIndex.from_arrays(columnArray, names = ['Delta','mktStrike'])
    mktOptionPrice_date.columns = pd.MultiIndex.from_arrays(columnArray, names = ['Delta','mktStrike'])
    
    
    mktImpliedVol_date_List.append(mktImpliedVol_date)
    mktFwdPrice_date_List.append(mktFwdPrice_date)
    mktOptionPrice_date_List.append(mktOptionPrice_date)

## Prepare Model Data

In [10]:
modelImpliedVol = mktImpliedVol.copy(deep = True)
modelImpliedVol.iloc[:,:] = np.nan
modelOptionPrice = mktOptionPrice.copy(deep = True)
modelOptionPrice.iloc[:,:] = np.nan

## Calibration

In [11]:
#Set Parameters (N个p(T),N个eplison(T),N个Sigma(T),Q(T))
PARAMS = np.zeros((len(dateList), 3 * N + 1))
PARAMS


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [12]:
objective_dict = {}
for maturityChoice in range(len(dateList)):
    date =dateList[maturityChoice]
    fwd = fwdMap[date]
    T = ((date - valuationDate).days)/365
    r = interpolate_riskFree(T)
    #q = GetImpliedDividendYield(spotPrice, r, T, fwd)
    
    mktOptionPrice_date = mktOptionPrice_date_List[maturityChoice]
    print(date)
    objective_dict[date] = {}
    
    for trial in range(totalTrials):
        print("trial",trial)   
        params_initial = np.random.random((1,N * 3 + 1))[0]
        #minimize()
        result = minimize(Objective(params_initial, mktOptionPrice_date, date, returnType = 0),
                          params_initial,method='SLSQP',
                          bounds=Bounds(PARAMS, maturityChoice),
                          constraints=Constraints(params_initial, returnType = 0))
        params = result.x
        objectiveResult = Objective(params, mktOptionPrice_date, date, returnType = 1)
        objective_dict[date][trial] = [objectiveResult, params]
        print(objectiveResult,AllStrikeError(params, date, mktOptionPrice_date))
        #print(params)
        print("====="*5)

    #print(objective_dict[date])
    objective_dict[date] = sorted(objective_dict[date].items(), key = lambda x: x[1][0])
    params = objective_dict[date][0][1][1]
    PARAMS[maturityChoice] = params

    for i, columns in enumerate(mktOptionPrice_date):
        delta = columns[0]
        mktStrike = columns[1] 
        modelPrice = ModelPrice(params, date, mktStrike, call = (delta > 0))
        modelOptionPrice.iloc[maturityChoice,i] = modelPrice
        modelImpliedVol.iloc[maturityChoice,i] = SolveIV(spotPrice, mktStrike, r, T, q, modelPrice, call = (delta > 0))
        
    print("#"*50)

2021-09-21 00:00:00
trial 0
9.540843027175176e-05 0.07740720072098435
trial 1
9.22208236537777e-06 0.002319168347885225
trial 2
4.879206286516188e-05 0.022083095510457246
trial 3
0.0008101960557013447 0.0031597941510744655
trial 4
0.005461226717983495 0.008035314745597336
trial 5
1.1790740532317522e-05 0.00286601938281737
trial 6
2.063014732400775e-05 0.0005178842273607077
trial 7
0.0011937273372149674 0.001931070026319505
trial 8
0.005389166254507328 0.007859885491655179
trial 9
0.005613928070667335 0.008234494191049085
trial 10
0.0055712837266504904 0.008119611670144126
trial 11
4.542727500188174e-05 0.0032065901036988156
trial 12
0.00531643900290773 0.007711377316847102
trial 13
2.1325470630289233e-05 0.03840727036770784
trial 14
9.256711226733487e-05 0.003981536536323523
trial 15
2.0453916401773786e-05 0.021244579277237215
trial 16
6.325769304996325e-05 0.02657122209680189
trial 17
1.765966572234511e-05 0.001660223350254368
trial 18
0.00034384038648345173 0.052284606722321596
trial

1.000268007664989e-05 0.0015213048606998927
trial 6
7.209817505833331e-05 0.0013845623795184514
trial 7
0.0036732201701719694 0.008258765114777558
trial 8
0.0008968662734738723 0.002570576226190255
trial 9
0.0066429649734907345 0.016304622010888448
trial 10
0.008205390029693763 0.021202723344817216
trial 11
1.1310923558218776e-05 0.0039663315832185195
trial 12
0.0035395898519813476 0.007833825980103754
trial 13
0.008006687482640084 0.020607897479705272
trial 14
0.15851533008197025 0.18025187314342234
trial 15
0.0030582624832448816 0.00604451631458866
trial 16
1.8423270839599493e-05 0.002782161535283906
trial 17
0.006579853956029765 0.016190839645866286
trial 18
0.15905307144167233 0.18089537199496233
trial 19
0.0001283581688635506 0.0023625140019796808
trial 20
7.700910964544935e-06 0.004640989011240829
trial 21
0.003675914212456392 0.008264974987736888
trial 22
1.6615032044080357e-05 0.0011598588138373978
trial 23
3.5549330034741763e-06 0.0050445836556646285
trial 24
0.004753054063485

5.24733411181577 5.945199937872697
trial 12
0.004654974528427899 0.014665847805647114
trial 13
5.247280863795375e-05 0.006782333515145614
trial 14
0.0063073343714347835 0.01977138060411531
trial 15
0.004521921201488601 0.014173143198354447
trial 16
0.0052721224646535515 0.016015158706644668
trial 17
0.00012634541762665885 0.027017822562849857
trial 18
8.875421083751052e-05 0.004125605614708254
trial 19
1.571622170477005e-05 0.01080434568371263
trial 20
0.006373551557683636 0.01956934006007628
trial 21
0.006244297419461066 0.01982457757731064
trial 22
0.00012494281425745474 0.03367142676728229
trial 23
1.3211407352483045e-05 0.005894554822266845
trial 24
0.005317557897630866 0.017424148401664234
trial 25
0.0043451542280169555 0.015305415043362684
trial 26
0.004398010988575096 0.014541622246160771
trial 27
0.004618785106510781 0.015322341615112426
trial 28
0.004384444009854035 0.014421139715230234
trial 29
0.0043492737546044335 0.014341022227748895
trial 30
0.004669959124233879 0.0153319

In [13]:
mktImpliedVol

,-0.1,-0.15,-0.25,-0.35,0.5,0.35,0.25,0.15,0.1
date,,,,,,,,,
2021-09-21,0.2232,0.2160,0.2096,0.2068,0.2049,0.2045,0.2049,0.2065,0.2088
2021-10-21,0.2001,0.1943,0.1880,0.1850,0.1830,0.1829,0.1842,0.1886,0.1940
2021-11-21,0.2058,0.2030,0.1995,0.1972,0.1954,0.1953,0.1961,0.1974,0.1981


## Result

In [14]:
modelOptionPriceDiff = (modelOptionPrice/mktOptionPrice - 1)
modelOptionPriceDiff.applymap(lambda x: format(x, '.2%'))

,-0.1,-0.15,-0.25,-0.35,0.5,0.35,0.25,0.15,0.1
date,,,,,,,,,
2021-09-21,0.00%,-0.17%,-0.00%,0.03%,0.00%,-0.14%,-0.00%,0.47%,-0.00%
2021-10-21,0.00%,-0.16%,0.00%,0.06%,0.00%,0.22%,0.00%,-1.09%,-0.00%
2021-11-21,-0.00%,-0.12%,-0.00%,0.08%,0.00%,1.13%,0.00%,-1.56%,-0.00%


In [15]:
modelImpliedVolDiff = (modelImpliedVol/mktImpliedVol - 1 )
modelImpliedVolDiff.applymap(lambda x: format(x, '.2%'))

,-0.1,-0.15,-0.25,-0.35,0.5,0.35,0.25,0.15,0.1
date,,,,,,,,,
2021-09-21,0.00%,-0.13%,-0.00%,0.05%,0.00%,-0.04%,-0.00%,0.08%,-0.00%
2021-10-21,0.00%,-0.29%,0.00%,0.35%,0.00%,0.04%,0.00%,-0.12%,-0.00%
2021-11-21,-0.00%,-0.40%,-0.00%,1.00%,0.00%,0.14%,0.00%,-0.14%,-0.00%


In [16]:
for maturityChoice in range(len(dateList)):
    date = dateList[maturityChoice]
    mktOptionPrice_date = mktOptionPrice_date_List[maturityChoice]
    print(date)
    params = PARAMS[maturityChoice]
    print("训练误差: %f, 总误差: %f" % (Objective(params, mktOptionPrice_date, date, returnType = 1),AllStrikeError(params, date, mktOptionPrice_date)))
    constraintResult = Constraints(params, returnType = 1)
    print("约束条件返回值：", constraintResult)
    print("约束条件是否满足：", all([abs(x) < 0.001 for x in constraintResult]))
    print("="*20)

2021-09-21 00:00:00
训练误差: 0.000009, 总误差: 0.002319
约束条件返回值： (8.046829202967842e-08, -1.1102230246251565e-16)
约束条件是否满足： True
2021-10-21 00:00:00
训练误差: 0.000004, 总误差: 0.005045
约束条件返回值： (-5.594144416587454e-07, 1.6608936448392342e-13)
约束条件是否满足： True
2021-11-21 00:00:00
训练误差: 0.000004, 总误差: 0.008665
约束条件返回值： (-4.275998276126458e-06, 4.571898415406395e-13)
约束条件是否满足： True


In [17]:
PARAMS

array([[3.10088876e-03, 9.64375805e-20, 9.77707254e-01, 1.27392819e-04,
        1.90626589e-02, 8.80909085e-02, 5.26181287e+00, 1.00205665e+00,
        1.65119968e-01, 1.04853293e+00, 2.93603548e-02, 1.83852320e-01,
        3.84575519e-02, 1.20838542e-02, 5.15597774e-02, 1.80555899e-06],
       [3.61811105e-02, 2.65706945e-02, 9.31151140e-01, 6.08062846e-03,
        7.89543587e-15, 1.03918398e+00, 8.34304163e-01, 1.00956792e+00,
        2.83269185e-02, 1.50114246e+00, 2.93622144e-02, 1.83852974e-01,
        5.92633999e-02, 2.14609775e-01, 5.36918181e-02, 1.64267412e-05],
       [5.38533267e-01, 3.01350803e-02, 2.85546092e-01, 2.71165746e-03,
        1.43034117e-01, 1.01447537e+00, 5.57860579e-01, 1.03398227e+00,
        1.32639329e-02, 9.89765632e-01, 7.05398190e-02, 2.80167294e-01,
        8.64805435e-02, 6.41229169e-01, 9.66259596e-02, 3.97872517e-05]])

In [18]:
now = str(datetime.now().date())
filename = f'result_{now}'
filterMask = [f for f in os.listdir(output_path) if f.startswith(f"result_{now}")]
if len(filterMask) > 0:
    num = len(filterMask) + 1
else:
    num = 1
filename = filename + f"_v{num}.xlsx"
file_path = os.path.join(output_path,filename)
writer = pd.ExcelWriter(file_path, engine = 'openpyxl')
modelOptionPriceDiff.to_excel(writer, "modelOptionPriceDiff")
modelImpliedVolDiff.to_excel(writer, "modelImpliedVolDiff")
pd.DataFrame(PARAMS).to_excel(writer, "PARAMS")
writer.save()
writer.close()